# surprise 모듈 작동 확인

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg
import numpy as np

# surprise 모듈 이용 데이터 추천
* 후보들이 뽑히면 그 데이터들을 가져와서 mf 실행
* 예측 rating 값이 높은 몇가지를 추천
* surprise 패키지 예측 못하면 그냥 평균정도 때리는듯

In [2]:
# db에 있는거 바로 dataframe으로 바꿈
conn = psycopg.connect(host="sulleong.site", dbname="sulleong", user="sulleong", password="Sulleong104**", port=5432)
sql = "SELECT * FROM review WHERE beer_id in (176, 161, 1432);"  # WHERE country IS NOT NULL;
reviews = sqlio.read_sql_query(sql, conn)
# reviews = beers.set_index('id')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_4216\1738570365.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews = sqlio.read_sql_query(sql, conn)


### surprise 패키지에서 dataset은 아이디, 아이템 아이디, 평점 순서로 해야됨

In [3]:
ratings = reviews[['member_id', 'beer_id','overall']]

In [4]:
from surprise import Reader, Dataset, KNNBasic, SVD, accuracy, BaselineOnly

# surprise에서 쓸수 있도록 데이터 변환 작업
reader = Reader(rating_scale=(1, 20))
data = Dataset.load_from_df(ratings, reader)

# 모델 parameter 지정, 유사도 함수 설정
sim_options = {
    'name': 'pearson', # msd, cosine, pearson 있음, 자세한건 surprise 문서 참조
    'user_based': False, # user를 base로 할것인지 item을 base로 할 것인지 결정.
}

# 최종 모델

bsl_options = {
    "method": "als",
    "learning_rate": 0.05,
    "n_epochs": 1000
}

# algo = KNNBasic(k=3, sim_options=sim_options)
# algo = SVD()
algo = BaselineOnly(bsl_options=bsl_options)


In [5]:
from surprise.model_selection import train_test_split, cross_validate

# 데이터 형태 안바꾸면 계산도 못하게함;;
trainset = data.build_full_trainset()
t, test = train_test_split(data, test_size=0.9)


algo.fit(trainset)

Estimating biases using als...


### 정확도 함 측정하라 했으니까 측정...
* 테스트할 데이터가 train 데이터에 포함되어있긴 한데 그냥함

In [6]:
from surprise import accuracy

pr = algo.test(test)
print(accuracy.rmse(pr))
print(accuracy.mae(pr))

In [11]:

print(algo.predict(uid=10004862, iid=2))
print(algo.predict(uid=10004862, iid=161))
print(algo.predict(uid=10004862, iid=1432))

user: 10004862   item: 2          r_ui = None   est = 15.43   {'was_impossible': False}
user: 10004862   item: 161        r_ui = None   est = 13.25   {'was_impossible': False}
user: 10004862   item: 1432       r_ui = None   est = 16.53   {'was_impossible': False}


In [9]:
a =[1,2,3,4]
print(f'afd ({str(a)[1:-1]})')

afd (1, 2, 3, 4)
